<center><h1 style="color:#D4AF37"> ⚡⚡ AEMR ⚡⚡</h1>

<img src = "https://images.squarespace-cdn.com/content/v1/551972d8e4b0d571edc2e2c8/8f3abd33-0916-4415-9b24-593eea1d1e3c/Screen+Shot+2022-04-20+at+12.41.24+PM.png">

**Setting Up SQLite Environment in Jupyter Notebook**

In [147]:
%%capture
!pip install ipython-sql sqlalchemy
import sqlalchemy
sqlalchemy.create_engine("sqlite:///AEMR.db")
%load_ext sql
%sql sqlite:///AEMR.db

In [148]:
%%js
require(['notebook/js/codecell'], function (codecell) {
    codecell.CodeCell.options_default.highlight_modes['magic_text/x-mssql'] = { 'reg': [/%?%sql/] };
    Jupyter.notebook.events.one('kernel_ready.Kernel', function () {
        Jupyter.notebook.get_cells().map(function (cell) {
            if (cell.cell_type == 'code') { cell.auto_highlight(); }
        });
    });
});

<IPython.core.display.Javascript object>

<h3 style="color:#D4AF37"> ⚡ Part I. Energy Stability & Market Outages ⚡ </h3>

Energy stability is one of the key themes the AEMR management team cares about. To ensure energy security and reliability, AEMR needs to understand the following:
<p>
<b>

<li> What are the most common outage types and how long do they tend to last? 
<li> How frequently do the outages occur? 
<li> Are there any energy providers which have more outages than their peers which may be indicative of being unreliable? 
    
<p>

<u style="color:Maroon"> Please note that throughout the entire case study, we are interested ONLY in the Outages where Status = Approved. We don't have any interest in Outages that were cancelled or not approved. This means your WHERE Clause will ALWAYS contain the field `Where Status = Approved` </u>

<h3 style="color:#D4AF37"> Step 1: Analyzing Outage Events Over 2016 and 2017 </h3> 

To better understand the patterns and trends in outage events, I utilized SQL to count the number of valid outage events, specifically focusing on those with a status of **Approved**. This analysis spans the periods of 2016 and 2017 and categorizes the outages based on their reasons: **Forced**, **Consequential**, **Scheduled**, and **Opportunistic**.

In [106]:
%%sql
SELECT
    Outage_Reason,
    SUM(CASE WHEN Year = 2016 THEN 1 ELSE 0 END) AS "2016",
    SUM(CASE WHEN Year = 2017 THEN 1 ELSE 0 END) AS "2017",
    SUM(1) AS Total_Valid_Outages
FROM
    AEMR_Outage_Table
WHERE
    Status = "Approved"
    AND Year IN (2016, 2017)
GROUP BY
    Outage_Reason;

 * sqlite:///AEMR.db
Done.


Outage_Reason,2016,2017,Total_Valid_Outages
Consequential,181,127,308
Forced,1264,1622,2886
Opportunistic Maintenance (Planned),106,102,208
Scheduled (Planned),380,320,700


**Insights:**

* Forced Outages: This category had the highest number of incidents in both years, with a notable increase from 2016 to 2017. This suggests that Forced outages were a significant issue and became even more prevalent in 2017.

* Scheduled Outages: This type also showed a significant number of incidents, though the number decreased slightly in 2017 compared to 2016. This could reflect better planning or fewer scheduled outages in the latter year.

* Consequential and Opportunistic Maintenance: These categories had fewer outages overall, with a slight decrease in the number of Consequential outages from 2016 to 2017. The Opportunistic Maintenance showed a marginal decrease as well.

<h3 style="color:#D4AF37"> Step 2: Analyzing Monthly Outage Trends </h3>

In this analysis, my goal was to investigate the monthly distribution of various outage types (Forced, Consequential, Scheduled, and Opportunistic Maintenance (Planned)) over the years 2016 and 2017. By grouping and ordering the outages by year, month, and type, I sought to uncover any patterns or increases in specific outage types over time.

**Step i) Summarizing Total Outages by Year and Month**

To start, I wrote a SQL query to calculate the total number of outages each month across both 2016 and 2017. This helped establish a high-level view of how outage occurrences fluctuated on a monthly basis.

**Why this approach?**

* Common Table Expression (CTE): I utilized a CTE (OutageTotals) to first aggregate the outages by year, month, and reason. This allowed me to efficiently organize and filter the data in the subsequent query.

* Grouping and Ordering: The final query grouped the results by Outage_Reason, Year, and Month, and ordered the results to identify any noticeable trends or changes in outage types across the two years. This method is particularly useful for spotting whether a specific type of outage was increasing or decreasing month-to-month.

In [109]:
%%sql
WITH OutageTotals AS (
SELECT Year, Month, Outage_Reason, COUNT(EventID) AS Total_Valid_Outages
FROM AEMR_Outage_Table
WHERE Status = "Approved"
    AND Year IN (2016, 2017)
GROUP BY Year, Month, Outage_Reason
)
SELECT
    Year,
    Month,
    Outage_Reason,
    SUM(Total_Valid_Outages) AS Total_Outages
FROM
    OutageTotals
GROUP BY
    Outage_Reason, Year, Month
ORDER BY
    Outage_Reason, Year, Month DESC, Total_Outages DESC;

 * sqlite:///AEMR.db
Done.


Year,Month,Outage_Reason,Total_Outages
2016,12,Consequential,9
2016,11,Consequential,25
2016,10,Consequential,15
2016,9,Consequential,1
2016,8,Consequential,6
2016,7,Consequential,23
2016,6,Consequential,12
2016,5,Consequential,36
2016,3,Consequential,7
2016,2,Consequential,23


**Insights**

The analysis revealed that Forced outages showed the most dramatic increase from 2016 to 2017, particularly in the latter months of the year. The other outage types had more stable trends, with occasional peaks likely related to specific operational events or maintenance schedules. This insight could be valuable for prioritizing further investigation into the causes behind the rise in forced outages. However, frequency is only one part of the picture. The duration of these outages is also critical.

In essence, shorter outages are less concerning, but longer outages pose a greater risk to our energy supply. To address this, we need to identify the energy providers responsible for these prolonged outages.

<h3 style="color:#D4AF37"> Step 3: Calculating Average Outage Duration </h3>

In this step, I set out to calculate two critical metrics for each Participant Code and Outage Type across the years 2016 and 2017:

1. Total Number of Outage Events
2. Average Duration of Outages (in days)

My goal was to capture a clear picture of outage occurrences and their durations, focusing only on records where the **Status = Approved**. This analysis provides insight into which participant codes and outage types experienced the most events and how long those outages typically lasted.

**Approach and Considerations**

* Duration Calculation: To calculate the average duration of outages, I used the difference between Start_Time and End_Time. Given that some dates might be reversed in the dataset, I applied the ABS() function to ensure all duration values are positive. The JULIANDAY() function was instrumental here, as it calculates the difference in days, including fractional parts, allowing for precise duration measurement.

* Rounding for Clarity: I rounded the average duration to two decimal places, making comparisons easier and more meaningful. This level of precision ensures that even slight differences in duration are visible, which is important for identifying trends or anomalies.

* Grouping and Ordering: The data was grouped by Participant_Code, Outage_Reason, and Year, and then ordered by the total number of outage events in descending order. This method highlights which participants and outage types were most affected, making it easier to spot any significant patterns over the two-year period.

In [112]:
%%sql
SELECT Participant_Code, Outage_Reason, Year,
    COUNT(*) AS Total_Number_Outage_Events,
    ROUND(AVG(ABS(JULIANDAY(End_Time) - JULIANDAY(Start_Time))), 2) AS Average_Outage_Duration_In_Days
FROM AEMR_Outage_Table
WHERE Year IN (2016, 2017) AND Status = 'Approved'
GROUP BY Participant_Code, Outage_Reason, Year
ORDER BY Total_Number_Outage_Events DESC;

 * sqlite:///AEMR.db
Done.


Participant_Code,Outage_Reason,Year,Total_Number_Outage_Events,Average_Outage_Duration_In_Days
AURICON,Forced,2017,490,0.07
GW,Forced,2016,317,0.38
GW,Forced,2017,227,1.06
AURICON,Forced,2016,208,0.07
AUXC,Forced,2016,206,0.08
MELK,Forced,2017,177,2.28
TRMOS,Forced,2017,172,0.42
MELK,Forced,2016,157,0.83
PUG,Forced,2017,135,0.25
AUXC,Forced,2017,120,0.02


<h3 style="color:#D4AF37"> Step 4. Classifying Participant Risk </h3>

The initial analysis pinpointed which participants experience the most frequent outages and which endure the longest downtime. This information is crucial, but the next logical step is to classify these participants based on their reliability, specifically their uptime metrics.

**Risk Classification Criteria**

To provide a clear and actionable understanding of participant reliability, I classified each participant according to the following risk levels:

* High Risk: Participants are, on average, unavailable for more than 24 hours (1 day).
* Medium Risk: Participants are, on average, unavailable between 12 and 24 hours.
* Low Risk: Participants are, on average, unavailable for less than 12 hours.

**Approach to Classification**

Given these criteria, I needed to devise a method to categorize each participant's risk level based on their average outage duration. The goal was to incorporate these classifications into our existing dataset, focusing on all approved outage types (Forced, Consequential, Scheduled, Opportunistic) across all participant codes for the years 2016 and 2017.

**Why this approach?**

* Using a CASE Statement: The CASE statement was the tool of choice here, allowing for the conditional logic required to assign each participant to a specific risk category based on their average outage duration.

* Leveraging CTEs for Organization: I utilized a Common Table Expression (CTE) to summarize the outage data by participant, making it easier to apply the risk classification logic in the final query.

In [114]:
%%sql
WITH Outage_Summary AS (
    SELECT Participant_Code, Outage_Reason, Year,
        COUNT(*) AS Total_Number_Outage_Events,
        ROUND(AVG(ABS(JULIANDAY(End_Time) - JULIANDAY(Start_Time))), 2) AS Average_Outage_Duration_In_Days
    FROM AEMR_Outage_Table
    WHERE Year IN (2016, 2017)
        AND Status = 'Approved'
    GROUP BY Participant_Code, Outage_Reason, Year
)
SELECT Participant_Code, Outage_Reason, Year, Total_Number_Outage_Events, Average_Outage_Duration_In_Days,
    CASE
        WHEN Average_Outage_Duration_In_Days > 24 THEN 'High Risk'
        WHEN Average_Outage_Duration_In_Days >= 12 AND Average_Outage_Duration_In_Days <= 24 THEN 'Medium Risk'
        ELSE 'Low Risk'
    END AS Risk_Classification
FROM Outage_Summary
ORDER BY Average_Outage_Duration_In_Days;

 * sqlite:///AEMR.db
Done.


Participant_Code,Outage_Reason,Year,Total_Number_Outage_Events,Average_Outage_Duration_In_Days,Risk_Classification
AUXC,Forced,2017,120,0.02,Low Risk
WGUTD,Forced,2016,2,0.02,Low Risk
EUCT,Forced,2017,3,0.03,Low Risk
PMC,Forced,2017,40,0.04,Low Risk
WGUTD,Consequential,2017,1,0.04,Low Risk
WGUTD,Opportunistic Maintenance (Planned),2017,2,0.05,Low Risk
MCG,Scheduled (Planned),2017,4,0.06,Low Risk
STHRNCRS,Opportunistic Maintenance (Planned),2017,1,0.06,Low Risk
TRMOS,Opportunistic Maintenance (Planned),2016,1,0.06,Low Risk
AURICON,Forced,2016,208,0.07,Low Risk


<h3 style="color:#D4AF37"> Step 5: Refining Risk Classification with Forced Outages </h3>

Given that Consequential, Opportunistic, and Planned outages are often scheduled or expected, they might not pose the same level of risk as Forced outages. Forced outages, being unplanned, are inherently more dangerous to the stability of the electricity grid. Therefore, it makes sense to refine our risk categories to focus specifically on Forced outages.

**Introducing Additional Risk Criteria**

To better capture the risk posed by these outages, I decided to incorporate two additional criteria into our classification:

* Outage Duration: This remains a key factor, as extended outages represent a significant risk.
* Total Number of Outage Events: Frequent outages, even if brief, can indicate underlying reliability issues.

**Updated Risk Classification Criteria**

Based on the above considerations, the new risk categories are as follows:

* High Risk: The participant is either unavailable for more than 24 hours (on average) or has more than 20 total outage events.
* Medium Risk: The participant is either unavailable between 12 and 24 hours (on average) or has between 10 and 20 total outage events.
* Low Risk: The participant is unavailable for less than 12 hours (on average) or has fewer than 10 total outage events.
* N/A: If the outage type is not Forced, then it doesn't factor into the risk classification.

In [116]:
%%sql
WITH Outage_Summary AS (
    SELECT Participant_Code, Outage_Reason, Year,
        COUNT(*) AS Total_Number_Outage_Events,
        ROUND(AVG(ABS(JULIANDAY(End_Time) - JULIANDAY(Start_Time))), 2) AS Average_Outage_Duration_In_Days
    FROM AEMR_Outage_Table
    WHERE Year IN (2016, 2017)
        AND Status = 'Approved'
    GROUP BY Participant_Code, Outage_Reason, Year
)
SELECT Participant_Code, Outage_Reason, Year, Total_Number_Outage_Events, Average_Outage_Duration_In_Days,
    CASE
        WHEN Outage_Reason != 'Forced' THEN 'N/A'
        WHEN Average_Outage_Duration_In_Days > 24 OR Total_Number_Outage_Events > 20 THEN 'High Risk'
        WHEN (Average_Outage_Duration_In_Days >= 12 AND Average_Outage_Duration_In_Days <= 24) OR (Total_Number_Outage_Events >+ 10 AND Total_Number_Outage_Events <= 20) THEN 'Medium Risk'
        WHEN Average_Outage_Duration_In_Days < 12 OR Total_Number_Outage_Events <10 THEN 'Low Risk'
        ELSE 'Unknown'
    END AS Risk_Classification
FROM Outage_Summary
ORDER BY Average_Outage_Duration_In_Days DESC;

 * sqlite:///AEMR.db
Done.


Participant_Code,Outage_Reason,Year,Total_Number_Outage_Events,Average_Outage_Duration_In_Days,Risk_Classification
WGUTD,Scheduled (Planned),2017,1,10.58,N/A
KORL,Scheduled (Planned),2016,20,6.96,N/A
MELK,Scheduled (Planned),2017,70,6.89,N/A
COLLGAR,Scheduled (Planned),2017,9,6.36,N/A
EUCT,Forced,2016,11,5.9,Medium Risk
ENRG,Scheduled (Planned),2017,37,4.96,N/A
ENRG,Scheduled (Planned),2016,29,4.85,N/A
MELK,Scheduled (Planned),2016,85,4.61,N/A
GW,Scheduled (Planned),2016,45,4.43,N/A
COLLGAR,Scheduled (Planned),2016,12,4.28,N/A


<h3 style="color:#D4AF37"> ⚡ Part II. Energy Losses & Market Reliability ⚡ </h3>

When an energy provider provides energy to the market, they are making a commitment to the market and saying; “We will supply X amount of energy to the market under a contractual obligation.” However, in a situation where the outages are Forced, the energy provider intended to provide energy but are unable to provide energy and are forced offline. <b style="color:Maroon">If many energy providers are forced offline at the same time it could cause an energy security risk that AEMR needs to mitigate. </b>

To ensure this doesn’t happen, the AEMR is interested in exploring the following questions:

<li> Of the outage types in 2016 and 2017, what percent were Forced Outage(s)?
<li> What was the average duration for a forced outage during both 2016 and 2017? Have we seen an increase in the average duration of forced outages?
<li> Which energy providers tended to have the largest number of forced outages?
    
<p>

<b> We'll examine these questions below. </b>
    
<img src = "https://media.istockphoto.com/id/1281821795/photo/market-stock-graph-and-information-with-city-light-and-electricity-and-energy-facility-banner.jpg?s=612x612&w=0&k=20&c=RSN5LqeMW28HW10aA190_DWR5YJ5tG2wixHFPBV3uZE=">

<h3 style="color:#D4AF37"> Step 6: Analyzing the Proportion of Forced Outages </h3>

Understanding the proportion of forced outages within the overall outage landscape is crucial for assessing grid reliability and identifying any emerging risks. Specifically, I aimed to calculate the percentage of outages that were Forced over the 2016-2017 period and observe whether there were any notable changes in this outage type.

In [120]:
%%sql
SELECT
    SUM(CASE WHEN Outage_Reason = 'Forced' THEN 1 ELSE 0 END) AS Total_Number_Forced_Outage_Events,
    COUNT(*) AS Total_Number_Outages,
    (SUM(CASE WHEN Outage_Reason = 'Forced' THEN 1 ELSE 0 END) * 100) / COUNT(*) AS Pct_Outage_Forced
FROM AEMR_Outage_Table
WHERE Year IN (2016, 2017)
    AND Status = 'Approved'
GROUP BY Year;

 * sqlite:///AEMR.db
Done.


Total_Number_Forced_Outage_Events,Total_Number_Outages,Pct_Outage_Forced
1264,1931,65
1622,2171,74


It's clear to see now that `Forced Outages` are problematic. Not only are they the only outage type that generates financial losses as the Outage is unplanned, it seems there is a number of Energy Participants who have been having a significantly high number of Outages.

**What's next?**

To address this challenge effectively, we need to conduct a more nuanced analysis by breaking it down into two levels: Macro and Micro. The total reveals the Overall Duration a participant is offline / has lost energy, however, it doesn't reveal how *frequently* this occurs. In other words, if we have one or two very big outages, it might contribute to very large totals.

However, perhaps an <b> average </b> can help identify how big these Outages might really be, spread across the year!

**Next Steps**

Let's shift our focus to the average duration of forced outages to identify the true scale of these disruptions.

<h3 style="color:#D4AF37"> Step 7: Understanding Outage Impact </h3>

My goal is to get a comprehensive view of the impact each participant and facility experiences from outages. I want to focus on three key metrics for each combination of participant_code and facility_code:

1. Total Number of Outages: I need to count all approved outage events for each participant and facility to determine how frequently they experience disruptions.
2. Total Duration in Days: By summing the duration of each outage, I can evaluate the total offline time for each participant and facility. This is rounded to two decimal places for precision.
3. Total Energy Lost: Calculating the sum of energy lost during these outages gives me an idea of the scale of disruption in terms of energy loss.

In [123]:
%%sql
SELECT
    COUNT(*) AS Total_Number_Outages,
    ROUND(SUM(ABS(JULIANDAY(End_Time) - JULIANDAY(Start_Time))), 2) AS Total_Outage_Duration_In_Days,
    SUM(Energy_Lost_MW) AS Total_Energy_Lost,
    Outage_Reason,
    Participant_Code,
    Facility_Code,
    Year
FROM AEMR_Outage_Table
WHERE Year IN (2016, 2017)
    AND Status = 'Approved'
GROUP BY Participant_Code, Facility_Code, Year
ORDER BY Total_Energy_Lost DESC, Year DESC;

 * sqlite:///AEMR.db
Done.


Total_Number_Outages,Total_Outage_Duration_In_Days,Total_Energy_Lost,Outage_Reason,Participant_Code,Facility_Code,Year
577,107.92,30134.036999999957,Forced,AURICON,AURICON_PNJ_U1,2017
273,544.21,27841.714000000004,Forced,MELK,MELK_G7,2016
402,329.81,25985.101,Forced,GW,BW1_GREENWATERS_G2,2016
270,314.21,24651.79199999999,Forced,GW,BW1_GREENWATERS_G2,2017
298,108.37,22109.62399999998,Forced,AURICON,AURICON_PNJ_U1,2016
263,897.17,19124.377000000008,Forced,MELK,MELK_G7,2017
107,88.35,14910.085000000001,Forced,PMC,PMC_AG,2016
142,192.9,10505.465,Forced,PJRH,PJRH_GT11,2016
146,160.65,10050.240999999998,Forced,PJRH,PJRH_GT11,2017
61,50.31,9295.302999999996,Consequential,PMC,PMC_AG,2017


<h3 style="color:#D4AF37"> Step 8: Average Duration and Energy Lost </h3>

In this part of my analysis, I'm focused on understanding the characteristics of forced outages—those unplanned disruptions that have a significant impact. The objective is to get a clearer picture of the impact of forced outages across different participants and facilities.

**Approach:**

1. Average Duration in Days: By calculating the average duration of forced outages, I can assess how prolonged these disruptions are on average. This value is rounded to two decimal places for better readability.

2. Average Energy Lost: This metric provides insights into the typical amount of energy lost during forced outages, giving an idea of the scale of the impact.

In [125]:
%%sql
SELECT
    ROUND(AVG(ABS(JULIANDAY(End_Time) - JULIANDAY(Start_Time))), 2) AS Avg_Duration_In_Days,
    AVG(Energy_Lost_MW) AS Avg_Energy_Lost,
    Outage_Reason,
    Participant_Code,
    Facility_Code,
    Year
FROM AEMR_Outage_Table
WHERE Year IN (2016, 2017)
    AND Status = 'Approved'
GROUP BY Participant_Code, Facility_Code, Year
ORDER BY Avg_Energy_Lost DESC;

 * sqlite:///AEMR.db
Done.


Avg_Duration_In_Days,Avg_Energy_Lost,Outage_Reason,Participant_Code,Facility_Code,Year
0.82,152.38201639344257,Consequential,PMC,PMC_AG,2017
1.71,147.82498113207546,Forced,COLLGAR,COLLGAR_WF1,2016
0.83,139.34658878504675,Forced,PMC,PMC_AG,2016
1.99,101.98430036630037,Forced,MELK,MELK_G7,2016
1.91,95.46551562500001,Forced,COLLGAR,COLLGAR_WF1,2017
1.16,91.3029333333333,Forced,GW,BW1_GREENWATERS_G2,2017
1.87,79.28701149425295,Opportunistic Maintenance (Planned),KORL,KORL_GT3,2016
0.36,74.19336912751672,Forced,AURICON,AURICON_PNJ_U1,2016
1.36,73.98214788732395,Forced,PJRH,PJRH_GT11,2016
3.41,72.71626235741448,Forced,MELK,MELK_G7,2017


<h3 style="color:#D4AF37"> Step 9: Analyzing Energy Lost Due to Forced Outages </h3>

As part of my analysis, I aimed to understand the financial impact of forced outages by examining two key metrics for each facility and participant over the 2016 and 2017 periods:

1. Average Energy Lost: This tells me the typical amount of energy lost during forced outages.
2. Total Energy Lost: This gives a cumulative view of energy lost for each facility and participant.
Additionally, I wanted to calculate the percentage of energy lost due to forced outages relative to the total energy lost across all facilities.

**Approach:**

1. Calculating Metrics: I first aggregated the data to find the average and total energy lost for each facility and participant. By focusing specifically on forced outages, I can pinpoint the impact of these unplanned disruptions.

2. Percentage Calculation: To get a sense of the relative impact, I calculated the percentage of total energy lost due to forced outages. This involves dividing the total energy lost by each facility and participant by the overall total energy lost and converting this ratio into a percentage.

3. Ordering and Limiting Results: By ordering the results by total energy lost in descending order, I can identify which participants and facilities have contributed the most to energy loss due to forced outages. Limiting the results to the top three helps focus on the most significant contributors.

In [127]:
%%sql
With CTE1 AS
(
    SELECT
    AVG(Energy_Lost_MW) AS Avg_Energy_Lost,
    SUM(Energy_Lost_MW) AS Total_Energy_Lost,
    Outage_Reason,
    Participant_Code,
    Facility_Code,
    Year
FROM AEMR_Outage_Table
WHERE
    Status = 'Approved'
    AND Outage_Reason = "Forced"
GROUP BY Outage_Reason, Participant_Code, Facility_Code, Year
)
SELECT
    ROUND(AVG_Energy_Lost, 2) AS Avg_Energy_Lost,
    ROUND(Total_Energy_Lost, 2) AS Total_Energy_Lost,
    ROUND((Total_Energy_Lost / (SELECT SUM(Total_Energy_Lost) FROM CTE1)) * 100, 2) AS Percentage_Energy_Lost,
    Outage_Reason,
    Participant_Code,
    Facility_Code,
    Year
FROM CTE1
ORDER BY Total_Energy_Lost DESC
LIMIT 3;

 * sqlite:///AEMR.db
Done.


Avg_Energy_Lost,Total_Energy_Lost,Percentage_Energy_Lost,Outage_Reason,Participant_Code,Facility_Code,Year
44.16,21639.55,14.21,Forced,AURICON,AURICON_PNJ_U1,2017
85.14,19326.56,12.69,Forced,GW,BW1_GREENWATERS_G2,2017
49.69,15751.38,10.34,Forced,GW,BW1_GREENWATERS_G2,2016


<h3 style="color:#D4AF37"> Step 10: Analyzing Energy Loss for Top Participants </h3>

After identifying the top contributors to energy loss—GW, MELK, and Auricon—I wanted to delve deeper into their specific outages. Here’s how I approached it:

1. Total Energy Lost: I calculated the total energy lost for each participant and facility across both years.

2. Top Outage Reason: For each participant and facility, I identified which Description_Of_Outage contributed the most to the total energy lost. This required ranking the outage descriptions to find the top contributor for each participant and facility.

3. Percentage of Energy Loss: I computed the percentage of energy lost due to the top outage reason relative to the overall energy lost for each participant. This helps in understanding the impact of specific outage types.

In [129]:
%%sql
WITH TopParticipants AS (
    SELECT
        Participant_Code,
        Facility_Code,
        Description_Of_Outage,
        SUM(Energy_Lost_MW) AS Total_Energy_Lost,
        RANK() OVER(PARTITION BY Participant_Code ORDER BY SUM(Energy_Lost_MW) DESC) AS Rank
FROM AEMR_Outage_Table
WHERE Participant_Code IN ('GW', 'MELK', 'AURICON')
    AND Status = 'Approved'
GROUP BY Participant_Code, Facility_Code, Description_Of_Outage
)
SELECT
    Participant_Code,
    Facility_Code,
    Description_Of_Outage,
    ROUND(Total_Energy_Lost, 2) AS Total_Energy_Lost,
    ROUND((Total_Energy_Lost / (SELECT SUM(Total_Energy_Lost) FROM TopParticipants)) * 100, 2) AS Percentage_Energy_Lost,
    RANK
FROM TopParticipants
WHERE Rank = 1
ORDER BY Participant_Code, Facility_Code;

 * sqlite:///AEMR.db
Done.


Participant_Code,Facility_Code,Description_Of_Outage,Total_Energy_Lost,Percentage_Energy_Lost,Rank
AURICON,AURICON_PNJ_U1,Full unit trip,6033.87,4.03,1
GW,BW1_GREENWATERS_G2,Operational Issues caused real time forced outage.,28687.54,19.14,1
MELK,MELK_G7,Safety Issues,1100.0,0.73,1


<center><h1 style="color:#D4AF37"> ⚡⚡ THAT'S A WRAP ⚡⚡</h1>
    
<img src = https://www.energymagazine.com.au/wp-content/uploads/2021/05/shutterstock_1888482466-e1620278098312.jpg>
